# 10K Analysis - Sub Question Query Engine

In [11]:
import nest_asyncio
nest_asyncio.apply()

In [1]:
from llama_index import SimpleDirectoryReader, LLMPredictor, ServiceContext, GPTVectorStoreIndex
from llama_index.response.pprint_utils import pprint_response
from langchain import OpenAI

/Users/suo/miniconda3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


configure LLM

In [2]:
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=-1, streaming=True))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

load data 

In [3]:
lyft_docs = SimpleDirectoryReader(input_files=["../data/10k/lyft_2021.pdf"]).load_data()
uber_docs = SimpleDirectoryReader(input_files=["../data/10k/uber_2021.pdf"]).load_data()

build indices

In [4]:
lyft_index = GPTVectorStoreIndex.from_documents(lyft_docs)

In [5]:
uber_index = GPTVectorStoreIndex.from_documents(uber_docs)

configure query engine

In [6]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)

In [7]:
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

build sub-question query engine

In [8]:
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

In [12]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine, 
        metadata=ToolMetadata(name='lyft_10k', description='Provides information about Lyft financials for year 2021')
    ),
    QueryEngineTool(
        query_engine=uber_engine, 
        metadata=ToolMetadata(name='uber_10k', description='Provides information about Uber financials for year 2021')
    ),
]

s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

In [16]:
response = s_engine.query('Compare and contrast the customer segments and geographies that grew the fastest')

Generated 4 sub questions.
[uber_10k] Q: What customer segments grew the fastest for Uber
[uber_10k] A: 
Uber's Delivery offering grew the fastest, as it allowed consumers to search for and discover local restaurants, order a meal, and either pick-up at the restaurant or have the meal delivered. Additionally, Delivery also includes offerings for grocery, alcohol and convenience store delivery as well as select other goods.
[uber_10k] Q: What geographies grew the fastest for Uber
[uber_10k] A: 
Uber experienced the fastest growth in Mobility Gross Bookings in five metropolitan areas—Chicago, Miami, and New York City in the United States, Sao Paulo in Brazil, and London in the United Kingdom. Additionally, Uber experienced strong competition in large metropolitan areas, which led to the offering of significant Driver incentives and consumer discounts and promotions in these large metropolitan areas.
[lyft_10k] Q: What customer segments grew the fastest for Lyft
[lyft_10k] A: 
The custome

In [18]:
print(response)


Uber experienced the fastest growth in Mobility Gross Bookings in five metropolitan areas—Chicago, Miami, New York City in the United States, Sao Paulo in Brazil, and London in the United Kingdom. The customer segments that grew the fastest for Uber were consumers searching for and discovering local restaurants, ordering meals, and either picking up at the restaurant or having the meal delivered. Additionally, Uber experienced strong competition in large metropolitan areas, which led to the offering of significant Driver incentives and consumer discounts and promotions in these large metropolitan areas.

Lyft experienced the fastest growth in riders and drivers. Riders are diverse and dynamic, representing all adult age groups and backgrounds, and drivers are active members of their communities. Lyft has grown rapidly in cities across the United States and in select cities in Canada. Additionally, Lyft has expanded its offerings to include access to a network of shared bikes and scoot